In [1]:
!pip install tqdm

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# -*- coding:utf-8 -*-
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
import torchvision
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from models import *
from dataset import *
from helpfun import *
import transforms as T
from torchinfo import summary

加载数据和设置超参数

In [3]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=0)
model =DeepLabV3_resnet50_stride16()
# model =DeepLabV3_resnet50_stride8()
# model =DeepLabV3_resnet34_stride16()
# model =DeepLabV3_resnet18_stride16()

model = model.cuda()

device="cuda"
lr=0.01
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=1e-4)
loss = nn.CrossEntropyLoss(ignore_index=255)

batch_size=20
num_workers=21
epoch=40
max_acc=0


name='DeepLabV3_resnet50_stride16' #实验名字

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
train_transforms = T.Compose([
            T.Resize(224),
            T.RandomHorizontalFlip(0.5),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ])
test_transforms = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ])

train_data = MyDataset(root=r'./data/', txt_file='./data/list/train_aug.txt'
                        ,transforms=train_transforms)
val_data = MyDataset(root=r'./data/', txt_file='./data/list/val.txt'
                      ,transforms=test_transforms)

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True,num_workers=num_workers)
test_loader = DataLoader(dataset=val_data, batch_size=1,num_workers=num_workers)

In [5]:
train_acc=[]
test_acc=[]
train_loss=[]
test_loss=[]
logger = get_logger('%s.log' % (name))
mk_model_dir(name)
logger.info("-----------------------------------relate parameters!-----------------------------------")
logger.info("初始学习率：%.3f"%(optimizer.param_groups[0]['lr']))#获取optimizer中的lr，保留3位小数，训练时打印（这时候的logger.info等同于print）并且记录在log中。
logger.info("优化方式：%s"%(str(optimizer).partition(' ')[0]))#打印optimizer输出为类名，类名中就会包含SGD、AdamW，通过字符串分割来获取。
logger.info("最大学习次数：%.3f"%(epoch))
logger.info("batch_size：%.3f"%(batch_size))
logger.info("使用的网络：%s"%(str(type(model)).partition("'")[-1].partition('.')[-1][:-2]))#模型net不同于optimizer，打印net不输出类名，而是输出整个网络结构。使用type(net)获取类名，在使用字符串分割来获取。
logger.info(summary(model, input_size=(batch_size, 3, 224, 224)))
logger.info(model)
logger.info("使用的损失函数：%s"%(loss))#损失函数较为特殊，直接通过变量名criterion 得到CrossEntropyLoss()。

logger.info("是否使用scheduler：%s"%(scheduler if 'scheduler' in dir() else 'False'))
logger.info("训练集使用的数据增强：")
logger.info('\n'.join(map(str,print_transforms(train_transforms))))
logger.info("测试集使用的数据增强：")
logger.info('\n'.join(map(str,print_transforms(test_transforms))))
logger.info("-----------------------------------start training!-----------------------------------")
for i in (range(epoch)):
    temp_acc,temp_loss=train(train_loader, model, optimizer, loss,device,i,epoch,lr)
    train_acc.append(temp_acc)
    train_loss.append(temp_loss)

    temp_acc,temp_loss=test(test_loader,loss, model,device)
    test_acc.append(temp_acc)
    test_loss.append(temp_loss)
    if temp_acc>=max_acc:
        max_acc=temp_acc
        torch.save(model.state_dict(),  './model/{}/best.pth'.format(name))
    print("Epoch: %d  Train Loss: %.3f, Acc: %.3f"%(i+1,train_loss[i],train_acc[i]))
    print("Epoch: %d  Test Loss: %.3f, Acc: %.3f\n"%(i+1,test_loss[i],test_acc[i]))
    logger.info("Epoch: %d  Test Loss: %.3f, Acc: %.3f\n"%(i+1,test_loss[i],test_acc[i]))
    torch.save(model.state_dict(), './model/{}/{}.pth'.format(name,epoch))

  2%|▏         | 10/530 [00:17<15:23,  1.78s/it] 


KeyboardInterrupt: 